## Gaussian Process Regression wit GPy

In [1]:
import GPy
import pandas as pd

### 1. Read the data

In [2]:
all_positions = pd.read_csv('data/positions_xyz_128px_full.csv', delimiter=',')
all_positions

,x,y,z
0,10.138330,3.169981,15.099980
1,10.133340,3.269852,15.199850
2,10.125010,3.369500,15.299500
3,10.113370,3.468816,15.398820
4,10.098430,3.567688,15.497690
...,...,...,...
25352,5.115520,4.872968,9.123255
25353,4.845301,5.128964,8.955405
25354,4.577969,5.373679,8.790463
25355,4.314191,5.604666,8.628434


In [3]:
all_red_ball_positions = pd.read_csv('data/red_ball_coordinates_128px_full.csv', delimiter=',')
all_red_ball_positions

,u0,v0,u1,v1,u2,v2,u3,v3,u4,v4,u5,v5,u6,v6
0,NaN,NaN,45.038462,83.307692,86.925926,39.333333,0.333333,41.500000,NaN,NaN,88.136364,82.727273,68.5,118.5
1,NaN,NaN,43.350000,83.850000,87.000000,40.111111,0.444444,42.555556,NaN,NaN,89.500000,84.500000,68.5,118.5
2,NaN,NaN,43.791667,84.666667,87.310345,41.034483,0.444444,43.444444,NaN,NaN,89.473684,85.631579,68.5,118.5
3,NaN,NaN,43.411765,85.705882,87.400000,41.400000,0.666667,44.500000,NaN,NaN,89.777778,86.500000,68.5,118.5
4,NaN,NaN,44.000000,86.285714,87.740741,42.000000,1.062500,45.437500,NaN,NaN,89.944444,87.222222,68.5,118.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25352,68.500000,118.166667,60.500000,69.000000,NaN,NaN,86.988235,82.435294,55.0,24.0,NaN,NaN,NaN,NaN
25353,68.500000,118.166667,60.500000,69.000000,NaN,NaN,92.730337,87.629213,55.0,24.0,NaN,NaN,NaN,NaN
25354,68.571429,118.857143,60.500000,69.000000,NaN,NaN,110.329545,101.409091,55.0,24.0,NaN,NaN,NaN,NaN
25355,68.571429,118.857143,60.500000,69.000000,NaN,NaN,104.370787,97.269663,55.0,24.0,NaN,NaN,NaN,NaN


In [4]:
xyz_positions = pd.read_csv('data/positions_xyz_128px_016.csv', delimiter=',')
xyz_positions = xyz_positions.iloc[4:]
red_ball_positions = pd.read_csv('data/red_ball_coordinates_128px_016.csv', delimiter=',')
red_ball_positions.shape,  xyz_positions.shape

((2132, 14), (2132, 3))

### 2. Handle Missing Values

In [5]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_all = imputer.fit_transform(all_red_ball_positions)
y_all = imputer.fit_transform(all_positions)
X = imputer.fit_transform(red_ball_positions)
y = imputer.fit_transform(xyz_positions)

### 3. Prepare the Data

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20285, 14), (5072, 14), (20285, 3), (5072, 3))

In [8]:
from sklearn.preprocessing import StandardScaler

scaler_x = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_x.fit_transform(X_train)
y_scaled = scaler_y.fit_transform(y_train)

### 4. Train the Model

In [9]:
kernel = GPy.kern.RBF(input_dim=14)
model = GPy.models.GPRegression(X_scaled, y_scaled, kernel=kernel)
model.optimize(messages=True)